In [ ]:
#%%
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from tqdm import tqdm
from joblib import Parallel, delayed

from mgcpy.hypothesis_tests.transforms import k_sample_transform
from mgcpy.independence_tests.dcorr import DCorr

In [ ]:
np.random.seed(8888)

n_sims = 100
n_samples = 300
n_components = 2
n_permutations = 100
size = (n_samples, n_components)

#%% mgcpy package
dcorr = DCorr(which_test="biased")
p_vals = np.zeros(n_sims)
def worker(i):
    sample1 = np.random.uniform(0.2, 0.7, size = size)
    sample2 = np.random.uniform(0.2, 0.7, size = size)

    sample, indicator = k_sample_transform(sample1, sample2)
    p, p_meta = dcorr.p_value(sample,
                             indicator,
                             replication_factor = n_permutations,
                             is_fast=False
    )
    return p
    
p_vals = Parallel(n_jobs=-2, verbose=10)(delayed(worker)(i) for i in range(n_sims))

In [ ]:
plt.style.use("seaborn-white")
sns.set_palette("deep")

plt.figure()
sns.distplot(p_vals)
plt.title("MGCPy DCorr, 2-sample under null, unbiased, not fast")
plt.xlabel("p-value")
plt.savefig("mgcpy_dcorr_profile.png", facecolor="w")